In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Function to standardize column names
def standardize_column_names(data):
    data['pickup_datetime'] = pd.to_datetime(data['lpep pickup datetime'])
    data['dropoff_datetime'] = pd.to_datetime(data['lpep dropoff datetime'])
    data.columns = data.columns.str.replace(' ', '_').str.lower()
    return data

# Function to remove duplicate rows
def eliminate_duplicates(data):
    data = data.drop_duplicates().copy()
    numeric_cols = data.select_dtypes(include=['int', 'float']).columns
    data[numeric_cols] = data[numeric_cols].abs()
    data = data.loc[~data.duplicated()]
    return data

# Function to handle various invalid cases
def clean_invalid_data(data):
    data = data.copy()
    # Remove negative fares with zero distance
    cond1 = (data['fare_amount'] < 0) & (data['payment_type'].isin(["Cash", "Credit Card"])) & (data['trip_distance'] == 0)
    data = data[~cond1]
    
    # Correct negative fares with non-zero distance
    cond2 = (data['fare_amount'] < 0) & (data['payment_type'].isin(["Cash", "Credit Card"])) & (data['trip_distance'] != 0)
    data.loc[cond2, data.select_dtypes(include=['int', 'float']).columns] = data.loc[cond2, data.select_dtypes(include=['int', 'float']).columns].abs()
    
    # Set specific columns to zero for 'No charge' cases
    cond3 = (data['payment_type'] == 'No charge') & (data['trip_distance'] == 0) & (data['fare_amount'] < 0)
    data.loc[cond3, ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']] = 0
    
    # Remove disputes with zero distance
    cond4 = (data['fare_amount'] < 0) & (data['payment_type'] == 'Dispute') & (data['trip_distance'] == 0)
    data = data[~cond4]
    
    # Recalculate total amount
    cols_to_sum = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge']
    data['total_amount'] = data[cols_to_sum].sum(axis=1)
    
    # Drop unnecessary columns
    data = data.drop(columns=['ehail_fee', 'congestion_surcharge'])
    return data


In [9]:
# Function to add trip duration columns
def add_trip_duration(data):
    data = data.copy()
    data['trip_duration'] = (data['dropoff_datetime'] - data['pickup_datetime']).dt.total_seconds()
    data['trip_duration_hours'] = data['trip_duration'] / 3600
    return data

# Function to fill missing values
def fill_missing_values(data):
    data = data.copy()
    data['passenger_count'] = data['passenger_count'].fillna(data['passenger_count'].median())
    pickup_hours = data['pickup_datetime'].dt.hour
    data.loc[(data['extra'].isnull()) & (pickup_hours.between(16, 19)), 'extra'] = 1
    data.loc[(data['extra'].isnull()) & (pickup_hours.between(20, 23)), 'extra'] = 0.5
    data['payment_type'] = data['payment_type'].fillna(data['payment_type'].mode()[0])
    return data

# Function to handle outliers in specified columns
def handle_outliers(data):
    data = data.copy()
    data = data[data['passenger_count'] <= 6]
    
    def cap_values(column, lower_quantile=0.1, upper_quantile=0.9):
        lower = column.quantile(lower_quantile)
        upper = column.quantile(upper_quantile)
        return np.clip(column, lower, upper)
    
    data['trip_distance'] = cap_values(data['trip_distance'])
    data = data[data['fare_amount'] <= 200]
    
    data['fare_amount'] = cap_values(data['fare_amount'])
    data['extra'] = cap_values(data['extra'])
    data['tip_amount'] = cap_values(data['tip_amount'])
    
    data = data[data['tolls_amount'] <= 50]
    data['tolls_amount'] = np.where(data['tolls_amount'] > 10, data['tolls_amount'].median(), data['tolls_amount'])
    data = data[data['improvement_surcharge'] != -0.3]
    
    # Recalculate total amount
    cols_to_sum = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge']
    data['total_amount'] = data[cols_to_sum].sum(axis=1)
    return data

# Function to categorize trips into weeks
def categorize_into_weeks(data, pickup_col='pickup_datetime'):
    data = data.copy()
    data[pickup_col] = pd.to_datetime(data[pickup_col])
    data['week_number'] = data[pickup_col].dt.strftime('%U').astype(int)
    data['date_range'] = data[pickup_col].dt.strftime('%Y-%m-%d') + ' to ' + (data[pickup_col] + pd.DateOffset(days=6)).dt.strftime('%Y-%m-%d')
    return data


In [10]:
# Function to encode categorical features
def encode_categorical(data):
    data = data.copy()
    data = pd.get_dummies(data, columns=['vendor', 'trip_type', 'payment_type', 'rate_type'])
    
    label_encoder = LabelEncoder()
    data['pickup_loc_encoded'] = label_encoder.fit_transform(data['pu_location'])
    data['dropoff_loc_encoded'] = label_encoder.fit_transform(data['do_location'])
    data['store_and_fwd_flag'] = label_encoder.fit_transform(data['store_and_fwd_flag'])
    return data

# Function to identify weekends
def identify_weekends(date_range):
    start_date, end_date = date_range.split(' to ')
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    return 'Weekend' if (start_date.dayofweek >= 5) or (end_date.dayofweek >= 5) else 'Weekday'

# Function to add day type feature
def add_day_type_feature(data):
    data = data.copy()
    data['day_type'] = data['date_range'].apply(identify_weekends)
    return data


In [11]:
# Main function to preprocess the dataset
def preprocess_dataset(data):
    data = standardize_column_names(data)
    data = eliminate_duplicates(data)
    data = clean_invalid_data(data)
    data = add_trip_duration(data)
    data = fill_missing_values(data)
    data = handle_outliers(data)
    data = categorize_into_weeks(data)
    data = encode_categorical(data)
    data = add_day_type_feature(data)
    return data

# Example usage
input_file_path = 'C:/Users/marya/OneDrive/Desktop/Classification_project1/green_tripdata_2015-07_trimmed.csv'
trip_data = pd.read_csv(input_file_path)
cleaned_trip_data = preprocess_dataset(trip_data)
cleaned_trip_data.to_csv('C:/Users/marya/OneDrive/Desktop/Classification_project1/preprocessed_green_tripdata_2015-07.csv', index=False)

# Display cleaned data
cleaned_trip_data.head()


,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,pu_location,do_location,passenger_count,trip_distance,fare_amount,extra,mta_tax,...,payment_type_Uknown,rate_type_Group ride,rate_type_JFK,rate_type_Nassau or Westchester,rate_type_Negotiated fare,rate_type_Newark,rate_type_Standard rate,pickup_loc_encoded,dropoff_loc_encoded,day_type
0,2015-07-01 00:12:55,2015-07-01 00:16:53,0,"Brooklyn,Williamsburg (South Side)","Brooklyn,Bushwick South",1.0,0.88,5.0,0.5,0.5,...,False,False,False,False,False,False,True,101,54,Weekday
1,2015-07-01 00:38:35,2015-07-01 00:49:22,0,"Brooklyn,Williamsburg (South Side)","Queens,Ridgewood",1.0,3.00,11.0,0.5,0.5,...,False,False,False,False,False,False,True,101,222,Weekday
2,2015-07-01 00:59:01,2015-07-01 01:04:10,0,"Brooklyn,Bushwick North","Brooklyn,East Williamsburg",1.0,1.61,6.5,0.5,0.5,...,False,False,False,False,False,False,True,53,71,Weekday
3,2015-07-01 00:48:41,2015-07-01 00:54:24,0,"Brooklyn,Brownsville","Brooklyn,East Flatbush/Remsen Village",1.0,0.68,5.0,0.5,0.5,...,False,False,False,False,False,False,True,52,68,Weekday
4,2015-07-01 00:56:28,2015-07-01 01:07:45,0,"Brooklyn,East Flatbush/Remsen Village","Brooklyn,Brownsville",1.0,0.68,8.0,0.5,0.5,...,False,False,False,False,False,False,True,68,52,Weekday


In [ ]:
# file_path = 'C:/Users/marya/OneDrive/Desktop/Classification_project1/green_tripdata_2015-07_trimmed.csv'
# data = pd.read_csv(file_path)